In [2]:
import pandas as pd
import json
import glob
import os


In [6]:
data_path = os.path.join("..", "data", "*.json")
files = glob.glob(data_path)
files


['../data/Streaming_History_Audio_2024-2025_1.json',
 '../data/Streaming_History_Video_2023-2025.json',
 '../data/Streaming_History_Audio_2025_2.json',
 '../data/Streaming_History_Audio_2021-2024_0.json']

In [7]:
dfs = []

for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        dfs.append(pd.DataFrame(data))

df = pd.concat(dfs, ignore_index=True)
df.head()


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2024-07-06T13:38:48Z,ios,4899,GB,92.40.218.69,Ogo,Seyi Vibez,Ogo,spotify:track:1zM2WWrMOt5Jt2jR39cBDp,None,...,None,None,None,clickrow,endplay,True,True,False,1.720271e+09,False
1,2024-07-06T13:40:28Z,ios,99822,GB,92.40.218.69,C****l Bag,Rsko,Memory,spotify:track:1iglyuVDpikOnQHULBu1ss,None,...,None,None,None,clickrow,endplay,True,True,True,1.720273e+09,False
2,2024-07-06T13:40:29Z,ios,1416,GB,92.40.218.69,La vie d'un... / Ma préférée,Dadju,HÉRITAGE,spotify:track:3IedXlFglIwo754rxOID4x,None,...,None,None,None,clickrow,endplay,True,True,True,1.720273e+09,False
3,2024-07-06T13:41:35Z,ios,65131,GB,92.40.218.69,Inviolable,Popcaan,Inviolable - Single,spotify:track:6hix0bbbr6iSUfPlTGwlTv,None,...,None,None,None,clickrow,endplay,True,True,True,1.720273e+09,False
4,2024-07-06T13:44:30Z,ios,177000,GB,92.40.218.69,Ogo,Seyi Vibez,Ogo,spotify:track:1zM2WWrMOt5Jt2jR39cBDp,None,...,None,None,None,clickrow,trackdone,True,False,True,1.720273e+09,False


In [20]:
df_clean = df.rename(columns={
    'ts': 'timestamp',
    'master_metadata_album_artist_name': 'artist_name',
    'master_metadata_track_name': 'track_name',
    'master_metadata_album_album_name': 'album_name'
})

df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

df_clean['month'] = df_clean['timestamp'].dt.to_period('M')

df_clean = df_clean[df_clean['artist_name'].notnull()]

monthly = (
    df_clean.groupby(['month', 'artist_name'])['ms_played']
    .sum()
    .reset_index()
)

monthly.head()


/var/folders/fn/jg3vx9rd081c2p7hnwnbr_8w0000gn/T/ipykernel_9922/4278385733.py:10: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_clean['month'] = df_clean['timestamp'].dt.to_period('M')


,month,artist_name,ms_played
0,2021-07,Dave,51754
1,2023-02,Cruel Santino,110313
2,2023-02,DJ Kamol 2,49408
3,2023-02,Kid MARLEY,53290
4,2023-02,Poco Lee,8042


In [21]:
top_artists = (
    monthly.groupby('artist_name')['ms_played']
    .sum()
    .sort_values(ascending=False)
    .head(20)          # Top 20 biggest
    .index
)

top_artists


Index(['Burna Boy', 'Wizkid', 'Tiakola', 'Chris Brown', 'Fally Ipupa', 'Rsko',
       'AMARIA BB', 'Seyi Vibez', 'Rema', 'Aya Nakamura', 'Ya Levis', 'J Hus',
       'Azanti', 'Oxlade', 'Jacquees', 'Haile', 'Vianni', 'Tory Lanez',
       'Eric IV', 'Merveille'],
      dtype='object', name='artist_name')

In [23]:
pip install prophet --no-build-isolation


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 10.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s  0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 4/6 [cmdstanpy]resources]  WARNING: The scripts install_cmdstan and install_cxx_toolchain are installed in '/Library/Frameworks/Python.framework/Versions/3.11/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [prophet]m5/6 [prophet]
Note: you may need to restart the kernel to use updated packages.


In [26]:
from prophet import Prophet
import numpy as np

def forecast_artist(artist_name, monthly_df):
    # Extract data for the artist
    artist_df = monthly_df[monthly_df['artist_name'] == artist_name].copy()
    
    # rename for Prophet
    artist_df = artist_df.rename(columns={'month': 'ds', 'ms_played': 'y'})
    
    # Prophet requires actual datetime, not Period
    artist_df['ds'] = artist_df['ds'].astype(str)
    
    # Train Prophet model
    m = Prophet()
    m.fit(artist_df)
    
    # Predict for next 12 months
    future = m.make_future_dataframe(periods=12, freq='M')
    forecast = m.predict(future)
    
    # isolate the future months
    last = artist_df['ds'].max()
    future_pred = forecast[forecast['ds'] > last]
    
    # return summed prediction
    total_pred_ms = future_pred['yhat'].sum()
    return total_pred_ms


In [28]:
predictions = []

for artist in top_artists:
    try:
        total_future_ms = forecast_artist(artist, monthly)
        predictions.append((artist, total_future_ms))
    except Exception as e:
        print(f"Error for {artist}: {e}")

pred_df = pd.DataFrame(predictions, columns=['artist_name', 'predicted_ms'])
pred_df = pred_df.sort_values(by='predicted_ms', ascending=False)
pred_df


16:48:31 - cmdstanpy - INFO - Chain [1] start processing
16:48:31 - cmdstanpy - INFO - Chain [1] done processing
16:48:31 - cmdstanpy - INFO - Chain [1] start processing
16:48:31 - cmdstanpy - INFO - Chain [1] done processing
16:48:31 - cmdstanpy - INFO - Chain [1] start processing
16:48:31 - cmdstanpy - INFO - Chain [1] done processing
16:48:32 - cmdstanpy - INFO - Chain [1] start processing
16:48:32 - cmdstanpy - INFO - Chain [1] done processing
16:48:32 - cmdstanpy - INFO - Chain [1] start processing
16:48:32 - cmdstanpy - INFO - Chain [1] done processing
16:48:32 - cmdstanpy - INFO - Chain [1] start processing
16:48:32 - cmdstanpy - INFO - Chain [1] done processing
16:48:32 - cmdstanpy - INFO - Chain [1] start processing
16:48:32 - cmdstanpy - INFO - Chain [1] done processing
16:48:33 - cmdstanpy - INFO - Chain [1] start processing
16:48:33 - cmdstanpy - INFO - Chain [1] done processing
16:48:33 - cmdstanpy - INFO - Chain [1] start processing
16:48:33 - cmdstanpy - INFO - Chain [1]

,artist_name,predicted_ms
11,J Hus,2.988217e+08
2,Tiakola,9.308982e+07
6,AMARIA BB,7.274497e+07
1,Wizkid,6.205553e+07
5,Rsko,3.430251e+07
9,Aya Nakamura,3.369259e+07
10,Ya Levis,3.053526e+07
8,Rema,2.158818e+07
4,Fally Ipupa,1.915800e+07
15,Haile,1.782682e+07


In [29]:
predicted_total_ms_2026 = pred_df['predicted_ms'].sum()

predicted_minutes_2026 = predicted_total_ms_2026 / (1000 * 60)
predicted_minutes_2026


11135.936611116187

In [30]:
monthly_tracks = (
    df_clean.groupby(['month', 'track_name'])['ms_played']
    .sum()
    .reset_index()
)

top_tracks = (
    monthly_tracks.groupby('track_name')['ms_played']
    .sum()
    .sort_values(ascending=False)
    .head(30)
    .index
)


In [31]:
def forecast_track(track_name, monthly_tracks_df):
    track_df = monthly_tracks_df[monthly_tracks_df['track_name'] == track_name].copy()
    track_df = track_df.rename(columns={'month': 'ds', 'ms_played': 'y'})
    track_df['ds'] = track_df['ds'].astype(str)
    
    m = Prophet()
    m.fit(track_df)
    
    future = m.make_future_dataframe(periods=12, freq='M')
    forecast = m.predict(future)
    
    last = track_df['ds'].max()
    future_pred = forecast[forecast['ds'] > last]
    
    return future_pred['yhat'].clip(lower=0).sum()

track_predictions = []

for track in top_tracks:
    total_future_ms = forecast_track(track, monthly_tracks)
    track_predictions.append((track, total_future_ms))

track_df = pd.DataFrame(track_predictions, columns=['track_name','predicted_ms'])
track_df = track_df.sort_values(by='predicted_ms', ascending=False)
track_df.head(10)


17:00:31 - cmdstanpy - INFO - Chain [1] start processing
17:00:31 - cmdstanpy - INFO - Chain [1] done processing
17:00:32 - cmdstanpy - INFO - Chain [1] start processing
17:00:32 - cmdstanpy - INFO - Chain [1] done processing
17:00:32 - cmdstanpy - INFO - Chain [1] start processing
17:00:32 - cmdstanpy - INFO - Chain [1] done processing
17:00:32 - cmdstanpy - INFO - Chain [1] start processing
17:00:32 - cmdstanpy - INFO - Chain [1] done processing
17:00:32 - cmdstanpy - INFO - Chain [1] start processing
17:00:36 - cmdstanpy - INFO - Chain [1] done processing
17:00:36 - cmdstanpy - INFO - Chain [1] start processing
17:00:36 - cmdstanpy - INFO - Chain [1] done processing
17:00:36 - cmdstanpy - INFO - Chain [1] start processing
17:00:37 - cmdstanpy - INFO - Chain [1] done processing
17:00:37 - cmdstanpy - INFO - Chain [1] start processing
17:00:37 - cmdstanpy - INFO - Chain [1] done processing
17:00:37 - cmdstanpy - INFO - Chain [1] start processing
17:00:37 - cmdstanpy - INFO - Chain [1]

,track_name,predicted_ms
9,Peckham,3.268709e+08
17,Lady of Neptune,1.471569e+08
23,Like to Party,4.495291e+07
0,Tous les jours,2.922754e+07
3,Recognise,1.803905e+07
25,Karma,1.748488e+07
4,Masculine (feat. Burna Boy),1.643581e+07
20,Woman's Worth,1.544272e+07
12,G.A.N.G,1.194294e+07
26,Si j'savais,7.705696e+06


In [36]:
monthly_albums = (
    df_clean.groupby(['month', 'album_name'])['ms_played']
    .sum()
    .reset_index()
)

total_albums = (
    monthly_albums.groupby('album_name')['ms_played']
    .sum()
    .sort_values(ascending=False)
)

top_albums = total_albums.head(10)
top_albums


album_name
BDLM VOL.1        69796636
I Told Them...    51164460
Morayo            45597714
Memory            44154615
Outside           37668862
DNK               35630902
Formule 7         32813052
Droit Chemin      32346887
9PM IN PARIS      27373865
Made In Lagos     26345678
Name: ms_played, dtype: int64

In [38]:
df_clean.to_csv("../data/listening_history.csv", index=False)
